# Process wheel data into movements 

In [1]:
""" 
IMPORTS
"""
import os
import autograd.numpy as np
import pickle
from collections import defaultdict
import pandas as pd
from scipy.ndimage import gaussian_filter1d
from scipy import stats
from datetime import datetime
from scipy.interpolate import interp1d
from numba import njit, prange 
from one.api import ONE
import concurrent.futures

# Get my functions
functions_path =  '/home/ines/repositories/representation_learning_variability/Functions/'
functions_path = '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/Functions/'
os.chdir(functions_path)
from data_processing import process_quiescence
from wheel_functions import wheel_trial_epoch
one = ONE(base_url='https://alyx.internationalbrainlab.org')

/opt/anaconda3/envs/iblenv/lib/python3.10/site-packages/one/api.py:1577: UserWarning: Newer cache tables require ONE version 2.7 or greater
  warnings.warn(f'Newer cache tables require ONE version {min_version} or greater')


## Load training data

In [2]:
data_path =  '/home/ines/repositories/representation_learning_variability/DATA/Sub-trial/Design matrix/'
data_path =  '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/DATA/Sub-trial/Design matrix/'
os.chdir(data_path)
session_all = pickle.load(open('training_2024-04-23', 'rb'))

In [3]:
# Choose a session with good QC
data_path = '/home/ines/repositories/representation_learning_variability/Video and wheel/'
data_path = '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/Video and wheel/'
# save_path = '/home/ines/repositories/representation_learning_variability/Exported figures/'

os.chdir(data_path)
pass_qc = pickle.load(open(data_path + "good_brainwide_sessions_metadata", "rb"))
pass_qc = pickle.load(open(data_path + "good_dlc_bwm_sessions_metadata", "rb"))
mouse_names = pass_qc['subject_nickname'].unique()

In [6]:
def process_wheel_movements(mouse_name):

    data_path =  '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/DATA/Trial_data/'
    os.chdir(data_path)

    training_session = 'last'
    threshold = 0.2 # Need to check if this makes sense
    min_period = 400 # This is to match the minimum length of quiescence periods
    bin_size = 0.05
    mouse_data = pickle.load(open('training_data_trials_'+ mouse_name, "rb"))

    if training_session == 'last':
        mouse_session_num = np.max(mouse_data['training_day'])
        use_data = mouse_data.loc[mouse_data['training_day'] >= mouse_session_num]
    else:
        use_data = mouse_data.loc[mouse_data['training_day'] == training_session]

    data_path =  '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/DATA/Wheel/'
    os.chdir(data_path)

    # Loop through sessions
    mouse_sessions = use_data['session'].unique()
    for s, session in enumerate(mouse_sessions):
        
        session_trials = use_data.loc[use_data['session']==session]
        session_trials['index'] = np.arange(0, len(session_trials))
        
        df = wheel_trial_epoch(one, session_trials, session, bin_size, threshold, min_period).dropna()  

        # Save to a unique file
        pickle.dump(df, open("wheel_movements_" + str(training_session) + '_' + session, "wb"))


def process_quiescence_parallel(mouse_name):

    data_path =  '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/DATA/Trial_data/'
    os.chdir(data_path)

    training_session = 'last'
    mouse_data = pickle.load(open('training_data_trials_'+ mouse_name, "rb"))

    if training_session == 'last':
        mouse_session_num = np.max(mouse_data['training_day'])
        use_data = mouse_data.loc[mouse_data['training_day'] >= mouse_session_num]
    else:
        use_data = mouse_data.loc[mouse_data['training_day'] == training_session]

    # Loop through sessions
    mouse_sessions = use_data['session'].unique()
    for s, session in enumerate(mouse_sessions):
        data_path =  '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/DATA/Wheel/'
        os.chdir(data_path)
        wheel_data = pickle.load(open('wheel_movements_' + str(training_session) + '_' + session, "rb"))
        processed_quiescence = process_quiescence(wheel_data)

        # Save to a unique file
        data_path =  '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/DATA/Wheel/processed/'
        os.chdir(data_path)
        pickle.dump(processed_quiescence, open("processed_quiescence_" + str(training_session) + '_' + session, "wb"))


def parallel_process_data(mouse_names, function_name):
    with concurrent.futures.ThreadPoolExecutor() as executor:

        # Process each chunk in parallel
        executor.map(function_name, mouse_names)


In [7]:
# Loop through animals
function_name = process_wheel_movements
# function_name = process_quiescence_parallel
# data_path =  '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/DATA/Wheel/'
# os.chdir(data_path)
parallel_process_data(mouse_names, function_name)

/opt/anaconda3/envs/iblenv/lib/python3.10/site-packages/one/util.py:444: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datasets['revision'] = [rel_path_parts(x)[1] or '' for x in datasets.rel_path]
